In [1]:
import stats_table_functions as stats

situated tables built...
spect tables built...

Games available:

pilot14
pilot02


# Welcome to STAC stats! 
Here we are currently building functions to let you explore the situated and non-situated versions of the stac corpus.
For each command, you can specify which part of the corpus you would like to apply it to. 

Example:

stats.edu_count( ) returns the segment counts in a game. Run the command with "all" to see all games, the name of a game, e.g. "pilot02", to see one game, or a list of game names ["pilot02", "pilot04"]


<font color='green'>
### stats.edu_count( )</font>
<break>
takes either a list of games or 'all' to see the corpus


In [3]:
stats.edu_count('all')

,spect EDUs,situated EEUs,situated EDUs
pilot14,336,969,338
pilot02,301,464,301


In [4]:
stats.edu_count(['pilot02', 'pilot14'])

,spect EDUs,situated EEUs,situated EDUs
pilot02,301,464,301
pilot14,336,969,338


In [7]:
stats.edu_count('pilot02')

,spect EDUs,situated EEUs,situated EDUs
pilot02,301,464,301


<font color='green'> 
### stats.relation_count( )</font>
<break>
Takes 3 parameters:
1. games: either a list of games or 'all' -*OR*- 'aggregate' to see aggregated counts between situated and spect versions
2. version: either 'situated', 'spect' or 'both'
3. relation types: a list of types or 'all'

In [5]:
stats.relation_count('all', 'both', 'all')

,Acknowledgement,Alternation,Anaphora,Background,Clarification_question,Comment,Conditional,Continuation,Contrast,Correction,Elaboration,Explanation,Narration,Parallel,Q-Elab,Question-answer_pair,Result,Sequence
pilot14_situated,42,4,4,2.0,25,88,0.0,223,16,17,55,17,1,3,10,113,458,210.0
pilot14_spect,37,4,4,0.0,19,55,0.0,40,18,10,18,15,1,7,5,65,17,0.0
pilot02_situated,51,9,5,1.0,20,25,9.0,133,8,7,39,7,4,1,13,104,230,89.0
pilot02_spect,48,9,5,1.0,8,26,9.0,34,7,8,22,10,5,3,17,83,7,0.0


In [6]:
stats.relation_count('aggregate', 'both', 'all')

,spect,situated,situated - spect
Result,24.0,688,664.0
Sequence,0.0,299,299.0
Continuation,74.0,356,282.0
Question-answer_pair,148.0,217,69.0
Elaboration,40.0,94,54.0
Comment,81.0,113,32.0
Clarification_question,27.0,45,18.0
Acknowledgement,85.0,93,8.0
Correction,18.0,24,6.0
Background,1.0,3,2.0
